# **Usando Python e dados abertos no Google Colab**

Bruno Gimenes Delphim

[*'Discente do curso de Biblioteconomia e Ciência da Informação - Campus USP-RP*](https://github.com/bgd94)

Neste notebook do Google Colab, eu vou mostrar como utilizar Python para acessar dados abertos do governo.

---

## Python

## catalogos-dados-brasil (**tableschema & datapackage**)
Conforme https://github.com/dadosgovbr/catalogos-dados-brasil/blob/master/scripts/uso/como-usar-com-o-pandas.ipynb

### Utilizando o terminal/prompt de comando para instalar pacotes Python externos

In [ ]:
pip install datapackage tableschema-pandas plotly plotly_express

#### Imprimindo versões instaladas

In [ ]:
pip freeze | grep -e datapackage -e tableschema -e plotly

### Lendo o pacote de dados (**plotly**)

In [ ]:
import pandas as pd

# Para trabalhar com Frictionless Data – frictionlessdata.io
from tableschema import Storage
from datapackage import Package

# Para visualização
import plotly_express as px
import plotly as py, plotly.graph_objects as go


# Tentar gravar no Panda e imprimir "true" se for bem-sucedido
output = "true"
try:
    url = 'https://github.com/dadosgovbr/catalogos-dados-brasil/raw/master/datapackage.json'

    # Conectar ao armazenamento Pandas
    storage = Storage.connect('pandas')

    # Carregar o pacote de dados da URL e salvar no armazenamento
    package = Package(url)
    package.save(storage=storage)

    # Se não houver exceções, imprimir "true"


except Exception as e:
    # Se ocorrer uma exceção, imprimir "false" e a mensagem de erro
    output = "false"
    print(e)

output

storage.buckets

type(storage['catalogos'])

storage['solucao']

storage['catalogos'].head()

tipo_solucao = storage['catalogos'].groupby('Solução').count()['URL'].rename('quantidade')
tipo_solucao

px.bar(
    pd.DataFrame(tipo_solucao).reset_index(),
    x = 'Solução',
    y = 'quantidade',
    color = 'Solução',
    color_discrete_sequence = py.colors.qualitative.Set2
)

poder = storage['catalogos'].groupby('Poder').count()['URL'].rename('quantidade')
poder

go.Figure(
    data=go.Pie(
        labels=poder.index,
        values=poder.values,
        hole=.4
    )
).show()

esfera = storage['catalogos'].groupby('Esfera').count()['URL'].rename('quantidade')
esfera

go.Figure(
    data=go.Pie(
        labels=esfera.index,
        values=esfera.values,
        hole=.4
    )
).show()

uf = storage['catalogos'].groupby('UF').count()['URL'].rename('quantidade')
uf

px.bar(
    pd.DataFrame(uf).reset_index(),
    x = 'UF',
    y = 'quantidade',
    color = 'UF',
    color_discrete_sequence = py.colors.qualitative.Set3
)

## excercicio-dados-sp (**frictionless**)


### Utilizando o terminal/prompt de comando para instalar pacotes Python externos

In [ ]:
pip install frictionless plotly plotly_express

#### Imprimindo versões instaladas

In [ ]:
pip freeze | grep -e frictionless -e plotly

### Lendo o pacote de dados

In [111]:
import pandas as pd
#from google.colab import drive
from frictionless import describe, Detector, extract, formats, Resource, validate
from pprint import pprint
import sqlite3

In [49]:
#drive.mount("/content/drive", force_remount=True)

In [50]:
def convert_excel_to_csv(path, file, extension):
  content = pd.read_excel(f"{path}/{file}.{extension}")

  content.to_csv(f"{file}.csv", mode = "a", index = None, header = True)

  return f"{file}.csv"

In [70]:
csv_file_path = convert_excel_to_csv("/content", "IDESP_ESCOLA_2022", "xlsx")

In [ ]:
# Descrevendo os dados

description = describe(csv_file_path)
pprint(description)

In [53]:
# Ajustar metadados caso necessário (ex.: field_missing_values, resource.schema.foreign_keys -> .yaml)

In [ ]:
# Extraindo dados

rows = extract(csv_file_path)
#pprint(rows)

first_or_default = list(rows.values())[0]
print(first_or_default)

In [ ]:
# Validando dados

report = validate(csv_file_path)
pprint(report)

In [96]:
# Carrega extensao SQL no ambiente

%load_ext sql

# Cria arquivo de banco de dados SQLite local -> PostgreSQL seria mais adequado
# para aplicativos de grande escala e necessidades de segurança avançadas.
%%sql
sqlite:///project.db

In [ ]:
# Cria recurso no formato 'Data Resource' a partir do arquivo .csv validado

resource = Resource(csv_file_path)

In [ ]:
# Indexa/carrega dados do recurso 'Data Resource' na tabela 'idesp' do banco de dados

resource.index('sqlite:///project.db', name='idesp', fast=True, fallback=True)
print(Resource('sqlite:///project.db', control=formats.sql.SqlControl(table='idesp')).extract())

In [ ]:
# Seleciona primeiros 3 registros da tabela 'idesp' no banco de dados

%%sql
select * from idesp limit 3;

In [ ]:
!pip install db-sqlite3

In [113]:
import pandas as pd
import sqlite3
from datetime import date

In [114]:
db = sqlite3.connect("project.db")

In [115]:
db = sqlite3.connect("project.db")
query = """
  select * from idesp limit 10
"""
df = pd.read_sql_query(query,db)
df.head(5)

,ANO,CODIGO_CIE,CODIGO_INEP,CODIGO_DIRETORIA,NOME_DIRETORIA,NOME_ESCOLA,NOME_MUNICIPIO,ANOS_INICIAIS,ANOS_FINAIS,ENSINO_MEDIO
0,2022,12,35000012,10101,NORTE 1,AYRES DE MOURA PROFESSOR,SAO PAULO,NaN,4.25,NaN
1,2022,24,35000024,10101,NORTE 1,GAVIAO PEIXOTO BRIGADEIRO,SAO PAULO,4.0,2.53,0.95
2,2022,36,35000036,10101,NORTE 1,JOAO SOLIMEO,SAO PAULO,NaN,2.33,1.09
3,2022,48,35000048,10101,NORTE 1,WALFREDO ARANTES CALDAS PROFESSOR,SAO PAULO,NaN,NaN,0.97
4,2022,59,35000059,10101,NORTE 1,SEBASTIAO DE OLIVEIRA GUSMAO PROFESSOR,SAO PAULO,NaN,NaN,1.28


## Referências
 - SANTAREM SEGUNDO, Jose Eduardo. *Data Science para Humanas, Introdução a análise de dados, modelos e algoritmos de Machine Learning*. 26 de março de 2024. Faculdade de Filosofia, Ciências e Letras de Ribeirão Preto, Ribeirão Preto, SP
 - https://www.youtube.com/dataprofessor (acesso em 03/04/2024)
 - SOBRENOME, Nome do professor. Nome da disciplina ou aula. Data da aula. Nome da instituição, Local.
 - https://github.com/dadosgovbr/catalogos-dados-brasil/blob/master/scripts/uso/como-usar-com-o-pandas.ipynb (acesso em 03/03/2024)
 - ChatGPT-4 (diversos prompts diferentes)
 - https://framework.frictionlessdata.io/ (acesso em 04/04/2024)
 - https://thivyapriyaa.medium.com/google-colab-series-setting-up-sqlite-de2aaaa02ec5 (acesso em 04/04/2024)
 - https://medium.com/@raftaarrashedin100/sqlite-connection-using-python-in-google-colab-6c8130fd672d (acesso em 05/04/2024)